In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

#make plots inline using jupyter magic
%matplotlib inline

import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn import datasets, linear_model, metrics

# Imports from Chapter 4 Lab
import matplotlib as mpl

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from IPython.display import Markdown as md  #enable markdown within code cell
from IPython.display import display, Math, Latex

### Utility Functions

In [2]:
def get_data_path(data_name="none", 
                  data_file_name="project_dataset.csv"):
    """  Gets the data path, specific to the project configuration.
    
    :return path to data
    """
    
    proj_dirs = os.getcwd().split("code")
    data_dir = os.path.join(proj_dirs[0], "data")
    data_path = os.path.join(data_dir,data_file_name)
    
    return data_path

# Sample a given number of points from the provided dataset. seed for repeatability
def get_dataFrame_subset(subset_size, df, seed):
    return df.sample(n=subset_size, random_state=seed)

### Pre-processing

In [3]:
file_name = get_data_path()
# usecols=['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Flow Bytes/s', 'Label']
original_df = pd.read_csv(file_name, sep=',', encoding='utf-8', usecols=[' Flow Duration', 
                ' Total Fwd Packets', ' Total Backward Packets', 'Flow Bytes/s', ' Label'])

display(original_df.head())
display(original_df.describe())

FileNotFoundError: File b'/tmp/CSCE 623_SP2019 in-class Day 03 - simple linear regression_student.ipynb/data/project_dataset.csv' does not exist

### Data Distribution

In [10]:
benign = 'BENIGN'
benign_df = original_df[original_df[' Label'] == benign]
display(benign_df.head())
display(benign_df.describe())

# Get number of malicious entries
malicious_df = original_df[original_df[' Label'] != benign]
display(malicious_df.head())
display(malicious_df.describe())


actual_df = get_dataFrame_subset(50000, benign_df, 1)
display(actual_df.head())
display(actual_df.describe())

,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Bytes/s,Label
0,113095465,48,24,174.0122825,BENIGN
1,113473706,68,40,212.2253767,BENIGN
2,119945515,150,0,0,BENIGN
3,60261928,9,7,108.7087688,BENIGN
4,269,2,2,1576208.178,BENIGN


,Flow Duration,Total Fwd Packets,Total Backward Packets
count,1.681860e+05,168186.000000,168186.000000
mean,1.254040e+07,15.178790,18.187423
std,3.212635e+07,1130.354261,1504.144975
min,-1.000000e+00,1.000000,0.000000
25%,1.900000e+02,1.000000,1.000000
50%,3.121300e+04,2.000000,2.000000
75%,4.911678e+05,4.000000,2.000000
max,1.200000e+08,200755.000000,270686.000000


,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Bytes/s,Label
12637,5185118,7,7,644.7297824,Web Attack � Brute Force
12643,5057374,10,7,2099.113097,Web Attack � Brute Force
12700,81,1,1,0,Web Attack � Brute Force
12712,5271123,7,5,455.1212332,Web Attack � Brute Force
12791,5020638,7,4,478.6244298,Web Attack � Brute Force


,Flow Duration,Total Fwd Packets,Total Backward Packets
count,2.180000e+03,2180.000000,2180.000000
mean,6.533719e+06,10.945413,5.281193
std,7.492580e+06,39.124039,20.156303
min,4.000000e+00,1.000000,0.000000
25%,5.177226e+06,3.000000,1.000000
50%,5.476074e+06,3.000000,1.000000
75%,5.783897e+06,3.000000,1.000000
max,7.020306e+07,212.000000,114.000000


,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Bytes/s,Label
163317,2,2,0,1040000000,BENIGN
9988,51367,1,1,233.61302,BENIGN
58239,30819,1,1,3893.701937,BENIGN
4940,5957135,3,1,2.014391146,BENIGN
163272,169,2,2,1408284.024,BENIGN


,Flow Duration,Total Fwd Packets,Total Backward Packets
count,5.000000e+04,50000.000000,50000.000000
mean,1.244384e+07,10.284740,11.065360
std,3.194552e+07,704.772366,817.582473
min,-1.000000e+00,1.000000,0.000000
25%,1.910000e+02,1.000000,1.000000
50%,3.114800e+04,2.000000,2.000000
75%,4.726235e+05,4.000000,2.000000
max,1.200000e+08,155812.000000,179415.000000
